In [23]:
import pandas as pd

In [24]:
import pickle

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [27]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)    
    return df

In [28]:
df_jan = read_dataframe('data/yellow_tripdata_2022-01.parquet')
df_feb = read_dataframe('data/yellow_tripdata_2022-02.parquet')

In [29]:
df_jan.shape, df_feb.shape

((2463931, 19), (2979431, 19))

In [30]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [31]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime

In [32]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [33]:
# Calculate the standard deviation of the 'Column1'
std_dev = df_jan['duration'].std()

# Print the standard deviation
print("Standard Deviation:", std_dev)

Standard Deviation: 46.44530513776802


In [34]:
df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)].shape[0]/df_jan.shape[0] * 100

98.27547930522405

In [35]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

In [36]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [37]:
categorical = ['PULocationID', 'DOLocationID']
df_jan[categorical] = df_jan[categorical].astype(str)
df_feb[categorical] = df_feb[categorical].astype(str)

In [38]:
dv = DictVectorizer()

In [39]:
train_dicts = df_jan[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [40]:
X_train.shape

(2421440, 2)

In [41]:
target = 'duration'
y_train = df_jan[target].values
y_val = df_feb[target].values

In [42]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [43]:
y_train_pred = lr.predict(X_train)

mean_squared_error(y_train, y_train_pred, squared=False)

8.920327827581442

In [44]:
y_test_pred = lr.predict(X_val)

mean_squared_error(y_val, y_test_pred, squared=False)

9.638272212087234